In [7]:


import json
import folium
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from bs4 import BeautifulSoup
from urllib.request import urlopen
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')

postal_codes = []
Borough = []
Neighborhood = []

for table in tables:
    rows = table.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')

        if len(cells) > 1:
            rank = cells[0]
            rr = rank.text.strip()
            if "M" in rr:
                postal_codes.append(rank.text.strip())
            else:
                continue

            country = cells[1]
            Borough.append(country.text.strip())

            rate = cells[2]
            Neighborhood.append(rate.text.strip())

            

postal_codes.pop()
Borough.pop()
Neighborhood.pop()

idx = list(range(len(postal_codes)))

df = pd.DataFrame(postal_codes, index=idx, columns = ['postal_codes'])
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df = df.groupby(['postal_codes','Borough'], as_index=False).agg(lambda x: ','.join(x))
mask = df['Neighborhood'] == "Not assigned"
df.loc[mask, 'Neighborhood'] = df.loc[mask, 'Borough']
df.shape

(103, 3)